In [ ]:
#The modules are basic modules we have been using this whole term 
%matplotlib inline
%matplotlib notebook
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import GPy, GPyOpt
import scipy, scipy.spatial
import random

#These modules need to be downloaded and installed as described in the appendix
import bayesian_pdes as bpdes
import bayesian_design as bd

The equation we are solving is: $$\Delta u = 0,$$ 
with the boundary conditions $u(0,y)=u(1,y)=u(x,0)=0$ and $u(x,1) = \sin(\pi  x)$. 

This has the solution: $$u(x,y) = \frac{1}{\sinh(\pi)} \sin(\pi x) \sinh(\pi y).$$

In [ ]:
#We define the exponential kernel using Sympy
x_1,x_2, y_1, y_2 = sp.symbols('x_1 x_2 y_1 y_2')

#This is the parameter that varies
length_scale = 0.3
#Exponetial kernel 
k = sp.exp(-((x_1-y_1)**2 + (x_2-y_2)**2) / (2*length_scale**2))

# defining the operators 
def A(f): return f.diff(x_2, x_2) + f.diff(x_1, x_1)
def Abar(f): return f.diff(y_2, y_2) + f.diff(y_1, y_1)
def Identity(f): return f

ops = [A, Identity]
ops_bar = [Abar, Identity]

In [ ]:
#This creates all the matrices need to be evaluated to calculate the mean and variance
op_cache = bpdes.operator_compilation.compile_sympy(ops, ops_bar, k, [[x_1, x_2], [y_1, y_2]])

The boundary function is $$u(x,1) = \sin(\pi  x)$$

In [ ]:
# Non-constat Boundary condition 
def boundary_fun(x): 
    return np.sin(np.pi * x)

In [ ]:
#This creates the boundary points 
obs_per_bdy = 4
# need some observations scattered on the bdy
bdy_locs = [
    [[0., y] for y in np.random.uniform(0,1,obs_per_bdy)],
    [[1., y] for y in np.random.uniform(0,1,obs_per_bdy)],
    [[x, 0.] for x in np.random.uniform(0,1,obs_per_bdy)],
    [[x, 1.] for x in np.random.uniform(0,1,obs_per_bdy)]
]
bdy_locs = np.concatenate(bdy_locs)

bdy_values = np.zeros((bdy_locs.shape[0], 1))

# Adding the non-constant boundary observations to the rest of the boundary observations
for i in range(4):
    bdy_values[i+12] = boundary_fun(bdy_locs[i+12][0])
    

bdy_obs = (bdy_locs, bdy_values)

In [ ]:
# positions of the observation points on the boundary
plt.figure(figsize=(5,5))
plt.scatter(bdy_locs[:,0], bdy_locs[:,1])
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
# random observation locations inside the space
interior_obs = 10
obs_locations = np.c_[np.random.rand(interior_obs, 1), np.random.rand(interior_obs, 1)]

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(obs_locations[:,0], obs_locations[:,1])
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
# defining the rhs of the PDE 
def rhs(pts):
    return (0.0*pts[:,1])[:,None]
obs_values = rhs(obs_locations)

#indexer = np.arange(len(obs_locations))
orig_locations = obs_locations.copy()
orig_values = obs_values.copy()
#this is the solver
orig_posterior = bpdes.collocate(ops, ops_bar, [(orig_locations, orig_values), bdy_obs], op_cache)

In [ ]:
#test points
plot_x, plot_y = np.mgrid[0:1:40j, 0:1:40j]

In [ ]:
#Evaluating the mean and cov at the test points 
mu, cov = orig_posterior(np.c_[plot_x.ravel(), plot_y.ravel()])

In [ ]:
#epistemic uncertainty 
sigma_epi = np.sqrt(np.diag(cov)).flatten()

In [ ]:
#to check whether the covariance matrix is symmetric
plt.matshow(cov); plt.colorbar();
plt.show()

In [ ]:
#This part can be skipped, it is just for sampling solutions from the posterior solution
samples = (np.random.multivariate_normal(mu.ravel(),cov,500)).T

new_samples = []
for i in range(500): 
    new_samples.append(samples[:,i].reshape(plot_x.shape))

In [ ]:
#Plotting the sampled solutions from the posterior solution 
fig, axs = plt.subplots(2,5, figsize=(15, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)

axs = axs.ravel()

for i in range(10):
    
    random_sample = random.randint(0,499)
    axs[i].contourf(plot_x, plot_y, new_samples[random_sample])
    axs[i].scatter(orig_locations[:,0], orig_locations[:,1], c='black', marker='x')
    axs[i].set_title(random_sample)

    
plt.xlim(0,1); plt.ylim(0,1);   
plt.colorbar
plt.show()

In [ ]:
#Plots the mean function 
from IPython.display import display
plt.figure(figsize=(6,5))

plt.contour(plot_x, plot_y, mu.reshape(plot_x.shape))
plt.scatter(orig_locations[:,0], orig_locations[:,1], c='black', marker='x')
plt.xlim(0,1); plt.ylim(0,1);
plt.colorbar()
plt.show()

In [ ]:
#defining the solution to the PDE which will be used for comparisson 
def solution(x,y):
    const = 1 / np.sinh(np.pi) 
    return const * np.sin(np.pi*x) * np.sinh(np.pi*y)

In [ ]:
#plotting the mean vs the true solution 
from IPython.display import display
plt.figure(figsize=(7,5))

h_truth = plt.contour(plot_x, plot_y, solution(plot_x, plot_y), alpha = 0.7, colors='g')
h_mean = plt.contour(plot_x, plot_y, mu.reshape(plot_x.shape), linestyles='dashed', colors='b')
plt.scatter(orig_locations[:,0], orig_locations[:,1], c='black', marker='x', label='Collocation points')
plt.xlim(0,1); plt.ylim(0,1); 
plt.legend(loc=3)
plt.xlabel('$x$', fontsize=14)
plt.ylabel('$y$', fontsize=14)
#plt.savefig('figures/2_D_laplace/2_D_sol_l03_var', transparent=True, bbox_inches=0)
plt.show()

In [ ]:
#plotting the confidence intervals
from IPython.display import display
plt.figure(figsize=(7,5))

plt.contour(plot_x, plot_y, mu.reshape(plot_x.shape) -2*sigma_epi.reshape(plot_x.shape), colors='r')
plt.contour(plot_x, plot_y, mu.reshape(plot_x.shape) +2*sigma_epi.reshape(plot_x.shape), colors='b', linestyles='dashed')

plt.scatter(orig_locations[:,0], orig_locations[:,1], c='black', marker='x', label='Collocation points')
plt.xlim(0,1); plt.ylim(0,1); 
plt.legend(loc=3)
plt.xlabel('$x$', fontsize=14)
plt.ylabel('$y$', fontsize=14)
#plt.savefig('figures/2_D_laplace/2_D_sol_l03_var', transparent=True, bbox_inches=0)
plt.show()

In [ ]:
#this is for the uncertainty calculations
int_results = {}
ns = 5*np.arange(1,20)
#seeding the random number so the same design points are used
np.random.seed(31)
for n_pts in 5*np.arange(1,20):
    # Creating the observed positions
    
    interior_obs = n_pts
    obs_locations = np.c_[np.random.rand(interior_obs, 1), np.random.rand(interior_obs, 1)]
    obs_values = rhs(obs_locations)
    
    # Using PMM to solve for observed values
    int_posterior = bpdes.collocate(ops, ops_bar, [(obs_locations, obs_values), bdy_obs], op_cache)    
    int_results[n_pts] = int_posterior(np.c_[plot_x.ravel(), plot_y.ravel()])

In [ ]:
#this is for the uncertainty calculations
int_errors = np.empty_like(ns, dtype=np.float)
int_traces = np.empty_like(ns, dtype=np.float)
actual = solution(plot_x, plot_y)
for ix, n in enumerate(ns):
    mu_int, cov_int = int_results[n]
    
    int_errors[ix] = np.linalg.norm(mu_int.reshape(plot_x.shape) - actual)
    
    int_traces[ix] = np.trace(cov_int)

In [ ]:
#plots the error in conditional mean
plt.semilogy(ns, int_errors, label='Integral Kernel', marker='o', markerfacecolor='white')
plt.ylabel('$\|\\mu - u \|$', fontsize=14)
plt.xlabel('$m_{\\mathcal{A}}$', fontsize=14)

#plt.savefig('figures/2_D_laplace/kernel_mean_error_l03', transparent=True, bbox_inches=0)
plt.legend()
plt.show()

In [ ]:
#plots the residual uncertainty 
plt.semilogy(ns, int_traces, label='Integral Kernel', marker='o', markerfacecolor='white')
plt.ylabel('$Tr(\\Sigma)$', fontsize=14)
plt.xlabel('$m_{\\mathcal{A}}$', fontsize=14)
#plt.savefig('figures/2_D_laplace/kernel_trace_l03', transparent=True, bbox_inches=0)
plt.legend()
plt.show()